<a href="https://colab.research.google.com/github/ISaySalmonYouSayYes/BA_thesis/blob/main/BERT_sentment_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Before you start**

In [1]:
try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

import os

num_cpus = os.cpu_count()
print(f"Number of available CPU cores: {num_cpus}")

Note: using Google CoLab
Number of available CPU cores: 12


In [ ]:
!pip install transformers datasets torch

# **1. Load training data(Amazon Review)**

In [3]:
from datasets import load_dataset

dataset_orig = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Health_and_Personal_Care")
print(dataset_orig) #<class 'datasets.dataset_dict.DatasetDict'>

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

The repository for McAuley-Lab/Amazon-Reviews-2023 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/McAuley-Lab/Amazon-Reviews-2023.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Health_and_Personal_Care.jsonl:   0%|          | 0.00/227M [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

DatasetDict({
    full: Dataset({
        features: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'],
        num_rows: 494121
    })
})


In [4]:
dataset = dataset_orig['full']
print(dataset[0]) #<class 'datasets.arrow_dataset.Dataset'>

{'rating': 4.0, 'title': '12 mg is 12 on the periodic table people! Mg for magnesium', 'text': 'This review is more to clarify someone else’s review bc they didn’t understand understand the labeling!  It shows 1000mg as advertised & another little label says 12mg bc 12 is on the periodic table for magnesium!  I realize not everyone takes chemistry, but 4 ppl liked his review & so misinformation is spreading.  This works. If however you are on opiate level medications that are causing constipation you should talk to your pain dr or your gastrointestinal dr & ask for a medication called Linzess which works must better & must faster, but is unnecessary for most people.  If magnesium is working for you just make sure to take it with food & drink 6-8 glasses of water per day.  Staying hydrated will really help.  Before switching to Linzess I used to take one 1,000 mg pill am & pm every day with meals & always with an 8 ounce glass of water or other liquid.', 'images': [], 'asin': 'B07TDSJZM

# **2. Preprocess training data**

In [5]:
import pandas as pd
from datasets import Dataset

# Split into train and test sets
dataset = dataset.train_test_split(test_size=0.2) #<class 'datasets.dataset_dict.DatasetDict'>
train_data, test_data = dataset['train'], dataset['test']

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Assign sentiment labels
def assign_sentiment_labels(example):
    if example['rating'] <= 2:
        example['rating'] = 0  # Negative
    elif example['rating'] >= 4:
        example['rating'] = 1  # Positive
    return example

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Assign labels
train_data = train_data.map(assign_sentiment_labels, num_proc=12)
test_data = test_data.map(assign_sentiment_labels, num_proc=12)

# Tokenize data
train_data = train_data.map(tokenize_function, batched=True, num_proc=12)
test_data = test_data.map(tokenize_function, batched=True, num_proc=12)

# Keep only 'input_ids', 'attention_mask', and 'rating'
columns_to_keep = ['input_ids', 'attention_mask', 'rating']
train_data = train_data.remove_columns([col for col in train_data.column_names if col not in columns_to_keep])
test_data = test_data.remove_columns([col for col in test_data.column_names if col not in columns_to_keep])

# Verify the dataset
print(train_data[0])
print(test_data[0])



Map (num_proc=12):   0%|          | 0/395296 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/98825 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/395296 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/98825 [00:00<?, ? examples/s]

{'rating': 1.0, 'input_ids': [101, 1045, 2066, 2009, 2062, 2084, 1045, 2130, 3517, 1012, 2009, 2003, 2235, 2021, 2008, 3084, 2009, 12109, 2065, 2017, 2215, 2000, 2202, 2009, 7118, 2030, 13215, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'rating': 1.0, 'input_ids': [101, 2844, 4487, 16643, 10820, 1012, 1045, 2031, 2018, 2009, 208

In [13]:
print(train_data.column_names)
train_data = train_data.rename_column("rating", "label")
test_data = test_data.rename_column("rating", "label")
print(train_data.column_names)

['rating', 'input_ids', 'attention_mask']
['label', 'input_ids', 'attention_mask']


# **3. Train the model**

In [16]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)  # 2 classes: pos/neg # Use num_labels=1 for binary classification

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Train the model
trainer.train()
results = trainer.evaluate()
print(results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# **4. Playground**

In [ ]:
import torch

text = "I love this product! It works perfectly."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

with torch.no_grad():
    logits = model(**inputs).logits

prediction = torch.argmax(logits, dim=-1).item()
label = "Positive" if prediction == 1 else "Negative"
print(f"Predicted Sentiment: {label}")

In [10]:
from transformers import BertTokenizer

with open("/content/data104.txt", "r") as file:
    file_content = file.read()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
file_content_token = tokenizer(file_content, truncation=True, padding="max_length", max_length=128)

print(file_content[:100])
print(file_content_token)

＠データ１０４（５４分）
＠収集年月日：２００２年１月１６日
＠場所：F106宅
＠参加者F106：女性１０代前半、オーストラリア生まれ、名古屋市在住
＠参加者F150：女性１０代前半、名古屋市出身、
{'input_ids': [101, 100, 100, 1987, 100, 1775, 1988, 100, 100, 100, 1840, 1872, 1864, 1993, 100, 1840, 100, 1872, 100, 1864, 100, 1806, 100, 1993, 20069, 2692, 2575, 100, 100, 100, 1779, 100, 20069, 2692, 2575, 1993, 1815, 100, 100, 1760, 1776, 100, 1635, 1699, 30265, 30233, 30240, 30257, 30258, 30219, 1910, 1677, 30214, 1635, 1795, 1789, 100, 100, 100, 100, 100, 100, 1779, 100, 20069, 12376, 1993, 1815, 100, 100, 1760, 1776, 100, 1635, 1795, 1789, 100, 100, 1774, 100, 1635, 1795, 1789, 100, 100, 100, 100, 100, 100, 1779, 100, 1671, 100, 100, 1993, 1746, 1817, 100, 1671, 1794, 100, 1910, 1636, 20069, 2692, 2575, 1993, 100, 100, 100, 1636, 20069, 12376, 1993, 1648, 30217, 1635, 1658, 30189, 30180, 30175, 100, 1647, 1636, 1643, 100, 1688, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0